In [1]:
from transformers import pipeline
from tqdm import tqdm
import random
import time

c:\Users\user\anaconda3\envs\MAIN_ENV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
unmasker = pipeline('fill-mask', model='xlm-roberta-large', device=0)
mask_token = "<mask>"
path_to_words_file = r"C:\Users\user\Desktop\AUTOENCODER\allwords.txt"

In [11]:
def generate_words(length):
    text = []
    
    with open(path_to_words_file) as file:
        words = file.read().split("\n")
    
    for x in range(length):
        text.append(words[random.randint(0, len(words) - 1)])
    
    return text

In [4]:
def get_skip_val(length):
    crnt_check = 1
    
    if length == 1:
        return 0
    
    while True:
        crnt_check += 1
        
        if crnt_check % length != 0 and length % crnt_check != 0:
            return crnt_check

In [22]:
def unmask_word_by_idx(topic_start, topic_end, completion, idx):
    completion  = completion.copy()
    completion[idx] = mask_token
    completion = " ".join(completion)
     
    prompt = f"{topic_start} {completion} {topic_end}"
    
    if topic_end == "":
        completion = unmasker(prompt)[0]["sequence"][len(topic_start):].split(" ")
    else:
        completion = unmasker(prompt)[0]["sequence"][len(topic_start):][:-len(topic_end)].split(" ")
    
    return completion

In [6]:
def add_length(topic_start, topic_end, completion):
    save = completion.copy()
    
    for x in range(len(completion)):
        completion = save.copy()
        start_length = len(completion) + 1
        insert_idx = x
        completion.insert(insert_idx, "")
        completion = unmask_word_by_idx(topic_start, topic_end, completion, insert_idx)
        
        if len(completion) > start_length:
            return completion
    
    return save + generate_words(1)

In [7]:
def subtract_length(topic_start, topic_end, completion):
    pop_idx = random.randint(0, len(completion) - 3)
    
    completion.pop(pop_idx)
    completion = unmask_word_by_idx(topic_start, topic_end, completion, pop_idx - 1)
    completion = unmask_word_by_idx(topic_start, topic_end, completion, pop_idx + 1)
    
    return completion

In [30]:
def complete(topic_start, topic_end, length, edurations, length_controll=False, length_controll_range=5):
    completion = generate_words(length)
    
    with tqdm(total=edurations) as pbar:  # Use tqdm as a context manager
        for x in range(edurations):
            updated_length = len(completion)
            skip = get_skip_val(updated_length)
            current_word = (x * skip) % updated_length - 1
            
            if length_controll:
                if updated_length < length - length_controll_range:
                    #print("underlength")
                    completion = add_length(topic_start, topic_end, completion)
                    
                if updated_length > length + length_controll_range:
                    #print("overlength")
                    completion = subtract_length(topic_start, topic_end, completion)
            
            completion = unmask_word_by_idx(topic_start, topic_end, completion, current_word)
            #print(completion)
            #TEMP_ghost = ["------"] * updated_length
            #TEMP_ghost[current_word] = mask_token
            #print(TEMP_ghost)
            
            pbar.update(1)  # Update the tqdm progress bar
    
    pbar.close()  # Close the tqdm progress bar after the loop has finished
    
    return " ".join(completion)

In [ ]:
conversation = ""

while True:
    conversation += f"\nquestion: {input('ask: ')}\nanswer:"
    conversation += complete(conversation, "", 6, 100)
    print(conversation)

In [ ]:
conversation = """question: hi! whats your name?
answer: my name is BERT.
question: whats your favourite color?
answer: blue.
question: whats your favourite food?
answer: my favourite food is pizza"""

while True:
    conversation += f"\nquestion: {input('ask: ')}\nanswer:"
    conversation += complete(conversation, ".", 10, 100)
    print(conversation)

In [ ]:
you = ["you said: ", "you replied: ", "you answered: ", "i questioned: ", "i mentioned: "]
i = ["i said: ", "i replied: ", "i answered: ", "i questioned: ", "i mentioned: "]

conv_start = f"""{you[random.randint(0, len(you) - 1)]}hi
{i[random.randint(0, len(i) - 1)]}hi how are you
{you[random.randint(0, len(you) - 1)]}im good thanks, and you?
{i[random.randint(0, len(i) - 1)]}im feeling alright overall."""

interaction = ""

conv_end = f"""{you[random.randint(0, len(you) - 1)]}alright thanks for telling me!
{i[random.randint(0, len(i) - 1)]}youre welcome.
{you[random.randint(0, len(you) - 1)]}its been nice talking to you
{i[random.randint(0, len(i) - 1)]}bye bye"""

while True:
    question = input("ASK QUESTION: ")
    interaction += f"you said: {question}"
    time.sleep(1)
    print(f"HUMAN: {question}")
    
    interaction += f"\n{i[random.randint(0, len(i) - 1)]}"
    answer = complete(f"{conv_start} {interaction}", conv_end, 10, 60)
    interaction += answer
    time.sleep(1)
    print(f"NRLNT: {answer}")
    
    interaction += f"\n{you[random.randint(0, len(you) - 1)]}"
    
    interaction = interaction[500:]